In [11]:
import sqlite3;
import pandas as pd;

#CONECTAR BASE DE DATOS

conexion = sqlite3.connect('datos_sensores.db');

query = """
SELECT
datos_basicos.tamaño,
datos_basicos.categoria,
orion.sensor_orion,
vega.sensor_vega,
polaris.sensor_polaris,
antares.sensor_antares,




CASE
  WHEN clasificacion.etiqueta = "Positivo" THEN 1
  ELSE 0
END AS etiqueta_numerica

FROM datos_basicos
INNER JOIN orion ON datos_basicos.id = orion.id
INNER JOIN vega ON datos_basicos.id = vega.id
INNER JOIN polaris ON datos_basicos.id = polaris.id
INNER JOIN antares ON datos_basicos.id = antares.id
INNER JOIN clasificacion ON datos_basicos.id = clasificacion.id
LIMIT 300000
"""

dataset = pd.read_sql_query(query,conexion);
dataset.head()

,tamaño,categoria,sensor_orion,sensor_vega,sensor_polaris,sensor_antares,etiqueta_numerica
0,0.02,3,19.77,42.15,35.36,-62.73,1
1,0.11,3,-23.51,-20.64,69.78,-38.02,1
2,0.85,1,86.63,75.26,30.82,-71.62,0
3,0.53,1,-44.91,4.11,71.92,27.17,0
4,0.14,2,-66.34,-48.04,-2.91,-74.93,0


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ... (código anterior para la conexión y carga de datos) ...

X = dataset.drop('etiqueta_numerica', axis=1)
Y = dataset['etiqueta_numerica']

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Normalización de datos
scaler = StandardScaler()
X_Train_scaled = scaler.fit_transform(X_Train)
X_Test_scaled = scaler.transform(X_Test)

# Manejo de desequilibrio de clases con SMOTE
smote = SMOTE(random_state=1)
X_Train_resampled, Y_Train_resampled = smote.fit_resample(X_Train_scaled, Y_Train)

# Modelo RandomForest
modelo_rf = RandomForestClassifier(random_state=1)
modelo_rf.fit(X_Train_resampled, Y_Train_resampled)

# Predicciones y evaluación del modelo
predicciones_rf = modelo_rf.predict(X_Test_scaled)

accuracy_rf = accuracy_score(Y_Test, predicciones_rf)
print(f"Precisión del Modelo RandomForest: {accuracy_rf:.2f}")

Reporte_rf = classification_report(Y_Test, predicciones_rf)
print(Reporte_rf)


Precisión del Modelo RandomForest: 0.90
              precision    recall  f1-score   support

           0       0.95      0.85      0.90     30309
           1       0.86      0.96      0.91     29691

    accuracy                           0.90     60000
   macro avg       0.91      0.90      0.90     60000
weighted avg       0.91      0.90      0.90     60000

